In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# pnu별 매출, 점포수, 유동인구, 거주인구 증감율
sql = 'select pnu, sale_12m, comp_12m, fltg_12m, lpop_12m, base_ym from m2.ibk_pnu_01'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df = pd.DataFrame(rows)
df.columns = ['pnu','sale_12m','comp_12m','fltg_12m','lpop_12m','base_ym']

# pnu별 주용도명
sql = '''
select main_purps_cd_nm, sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu from m1.bld_rgst
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_principal_use = pd.DataFrame(rows)
df_bld_principal_use.columns = ['principal_use','pnu']

In [ ]:
df

In [ ]:
df_bld_principal_use

In [ ]:
# 데이터 프레임 병합
df = pd.merge(df_f_tot, df_bld_rgst, on='pnu', how='left')

# 인구가 0명, null인 지역 제외
df = df[~df['bld_count'].isnull()]

# 상업용 부동산 수 / 유동인구
df['commercial_per_population'] = (df['bld_count'] / df['f_tot'])

# 주용도코드 붙이기
df = pd.merge(df, df_bld_principal_use, on='pnu', how='left')
df = df.drop_duplicates('pnu').reset_index(drop=True)

# inf 값 제거후 컬럼 정리
df = df[~(df['commercial_per_population']==np.inf)]
df = df[['pnu', 'principal_use', 'commercial_per_population']]
df['base_ym'] = datetime.today().strftime("%Y%m")

In [ ]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_gentrification'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
     # drop & create
    cursor.execute(
    """
    drop table if exists m2.datadam_gentrification;
    create table m2.datadam_gentrification (    
    pnu varchar(19)
    , principal_use varchar(200)
    , commercial_per_population double precision
    , base_ym varchar(6)
    )
    distkey(pnu)
    compound sortkey(principal_use, base_ym);
    commit;
    """
    )
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))